## make adjacency matrix to export

In [219]:
from pathlib import Path
import os, re
import numpy as np
import networkx as nx

btw, pathlib.Path is amazing, just started playing with it, fully object-oriented path manipulation

### set up paths, etc

In [220]:
%pwd

'/Users/alex/Dropbox/NeuroData/ndmg-top/exploration/discriminability'

In [221]:
%ls

JJ1~1~1~1~1~1~1~              output-explanation-master.md
README.md                     output-explanation-merge.md
desikan_res-1x1x1/            pr-diff.ipynb
discrim-graph.ipynb


In [222]:
p = Path('desikan_res-1x1x1')

In [223]:
# making sure I know what iterdir() does
[i for i in p.iterdir()][:5]

[PosixPath('desikan_res-1x1x1/sub-0025427_ses-9_dwi_desikan_res-1x1x1_measure-spatial-ds._elist.csv'),
 PosixPath('desikan_res-1x1x1/sub-0025441_ses-7_dwi_desikan_res-1x1x1_measure-spatial-ds._elist.csv'),
 PosixPath('desikan_res-1x1x1/sub-0025441_ses-1_dwi_desikan_res-1x1x1_measure-spatial-ds._elist.csv'),
 PosixPath('desikan_res-1x1x1/sub-0025427_ses-3_dwi_desikan_res-1x1x1_measure-spatial-ds._elist.csv'),
 PosixPath('desikan_res-1x1x1/sub-0025427_ses-5_dwi_desikan_res-1x1x1_measure-spatial-ds._elist.csv')]

In [224]:
num_subj = len([i for i in p.iterdir()])

### grab graphs

In [225]:
# list of numpy matrices
graphs = [nx.read_weighted_edgelist(str(i), delimiter=',') 
          for i in p.iterdir()]

### check nodelists, output sorted version to numpy matrix

In [226]:
len(graphs)

100

In [227]:
nx.to_numpy_array(graphs[0]).shape

(70, 70)

In [228]:
graphs[0].nodes()

NodeView(('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70'))

In [229]:
graphs[1].nodes()

NodeView(('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70'))

check the below list comprehension, because I'm not 100% confident it's right with edgelist sorting

In [230]:
graphs = [nx.to_numpy_array(g, nodelist=sorted(list(graphs[i].nodes())))
          for i, g in enumerate(graphs)]

### remove diagonals

In [231]:
graphs[0]

array([[1.8939e+05, 1.2863e+04, 4.4122e+04, ..., 2.5810e+03, 1.8606e+04,
        1.5608e+04],
       [1.2863e+04, 4.3680e+04, 2.2210e+03, ..., 1.2000e+01, 2.0859e+04,
        9.6090e+03],
       [4.4122e+04, 2.2210e+03, 5.3773e+04, ..., 2.5250e+03, 8.0040e+03,
        8.6920e+03],
       ...,
       [2.5810e+03, 1.2000e+01, 2.5250e+03, ..., 7.4160e+03, 1.2300e+02,
        7.0200e+02],
       [1.8606e+04, 2.0859e+04, 8.0040e+03, ..., 1.2300e+02, 4.8808e+04,
        5.8720e+03],
       [1.5608e+04, 9.6090e+03, 8.6920e+03, ..., 7.0200e+02, 5.8720e+03,
        4.8991e+04]])

In [232]:
for i, _ in enumerate(graphs):
    np.fill_diagonal(graphs[i], 0)

In [233]:
graphs[0]  # cool beans

array([[0.0000e+00, 1.2863e+04, 4.4122e+04, ..., 2.5810e+03, 1.8606e+04,
        1.5608e+04],
       [1.2863e+04, 0.0000e+00, 2.2210e+03, ..., 1.2000e+01, 2.0859e+04,
        9.6090e+03],
       [4.4122e+04, 2.2210e+03, 0.0000e+00, ..., 2.5250e+03, 8.0040e+03,
        8.6920e+03],
       ...,
       [2.5810e+03, 1.2000e+01, 2.5250e+03, ..., 0.0000e+00, 1.2300e+02,
        7.0200e+02],
       [1.8606e+04, 2.0859e+04, 8.0040e+03, ..., 1.2300e+02, 0.0000e+00,
        5.8720e+03],
       [1.5608e+04, 9.6090e+03, 8.6920e+03, ..., 7.0200e+02, 5.8720e+03,
        0.0000e+00]])

don't have to worry about nodelists getting removed here, cause networkx fills them in

### consider LCC

graphs is currently a list of numpy arrays, where each element is the per-subject adjacency matrix

In [234]:
# assert size of all matrices is 70x70
# problem: not all of these are 70x70
print(all(x.shape == (70, 70) for x in graphs), '\n')

for i, _ in enumerate(graphs):
    print(i, graphs[i].shape)
    
len(graphs)

True 

0 (70, 70)
1 (70, 70)
2 (70, 70)
3 (70, 70)
4 (70, 70)
5 (70, 70)
6 (70, 70)
7 (70, 70)
8 (70, 70)
9 (70, 70)
10 (70, 70)
11 (70, 70)
12 (70, 70)
13 (70, 70)
14 (70, 70)
15 (70, 70)
16 (70, 70)
17 (70, 70)
18 (70, 70)
19 (70, 70)
20 (70, 70)
21 (70, 70)
22 (70, 70)
23 (70, 70)
24 (70, 70)
25 (70, 70)
26 (70, 70)
27 (70, 70)
28 (70, 70)
29 (70, 70)
30 (70, 70)
31 (70, 70)
32 (70, 70)
33 (70, 70)
34 (70, 70)
35 (70, 70)
36 (70, 70)
37 (70, 70)
38 (70, 70)
39 (70, 70)
40 (70, 70)
41 (70, 70)
42 (70, 70)
43 (70, 70)
44 (70, 70)
45 (70, 70)
46 (70, 70)
47 (70, 70)
48 (70, 70)
49 (70, 70)
50 (70, 70)
51 (70, 70)
52 (70, 70)
53 (70, 70)
54 (70, 70)
55 (70, 70)
56 (70, 70)
57 (70, 70)
58 (70, 70)
59 (70, 70)
60 (70, 70)
61 (70, 70)
62 (70, 70)
63 (70, 70)
64 (70, 70)
65 (70, 70)
66 (70, 70)
67 (70, 70)
68 (70, 70)
69 (70, 70)
70 (70, 70)
71 (70, 70)
72 (70, 70)
73 (70, 70)
74 (70, 70)
75 (70, 70)
76 (70, 70)
77 (70, 70)
78 (70, 70)
79 (70, 70)
80 (70, 70)
81 (70, 70)
82 (70, 70)
83 (70,

100

### make single output graph

In [235]:
# for each element of graphs, flatten that element
# then, append that element as a row in the final output matrix

# no reason to keep using lists
graphs = np.array(graphs)

In [237]:
graphs.shape

(100, 70, 70)

In [238]:
graphs_final = np.reshape(graphs, (num_subj, 70*70))

In [239]:
graphs.shape

(100, 70, 70)

In [240]:
graphs_final.shape

(100, 4900)

In [241]:
check = np.reshape(graphs_final, (num_subj, 70, 70))

reshaping preserves information

In [242]:
np.all(check == graphs)

True

flattening is the same as reshaping on a single dimension basis

In [243]:
np.all(graphs[0].flatten() == graphs_final[0])

True

### save as csv

In [244]:
np.savetxt('BNU1_output_desikan.csv', graphs_final, fmt='%f', delimiter=",")

### get Y vector, save as csv

In [246]:
from collections import Counter

In [245]:
Y = [str(i) for i in p.iterdir()]

rgx = r'(sub-)([a-zA-Z0-9]*)'
Y = [re.findall(rgx, Y[i])[0][1] for i, _ in enumerate(Y)]
Y[:5]

['0025427', '0025441', '0025441', '0025427', '0025427']

In [249]:
# Check to make sure each subj has duplicates
Counter(Y)

Counter({'0025427': 5,
         '0025441': 5,
         '0025435': 5,
         '0025429': 5,
         '0025446': 5,
         '0025432': 5,
         '0025433': 5,
         '0025440': 5,
         '0025428': 5,
         '0025434': 5,
         '0025431': 5,
         '0025445': 5,
         '0025436': 5,
         '0025442': 5,
         '0025438': 5,
         '0025437': 5,
         '0025439': 5,
         '0025443': 5,
         '0025430': 5,
         '0025444': 5})

make Y vector file

In [250]:
for i in Y:
    with open("BNU1_output_Y.csv", "a") as f:
        f.write(i + '\n')